# Introduction

This notebook shows how to model, create, and use a Snowflake Semantic View on top of TPC-DS sample data. You will define the model in both SQL and YAML, convert between the two, run example queries, and integrate the model with Snowflake AI features.

## What you will do
- Prepare simple views over shared TPC-DS tables.
- Define a semantic model in YAML and SQL; validate and create a Semantic View.
- Convert between YAML and Semantic Views (read/write).
- Query the Semantic View using dimensions, metrics, and filters.
- Integrate with Cortex Analyst and enhance natural language understanding with Cortex Search.

## Prerequisites
- A Snowflake account and warehouse.
- Privileges to create databases, schemas, stages, and semantic views.

## Outcome
By the end, you will have a working Semantic View (`TPCDS_SEMANTIC_VIEW_SM`) and a repeatable workflow to manage it via YAML or SQL.

### Part 1: Prepare Data for a Meaningful Model

In this section, you will use publicly available Snowflake sample data (via a share) to create simple views that our Semantic View will reference later. This keeps the exercise focused on modeling rather than data loading.

In [ ]:
USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;

In [ ]:
-- Create or replace views for the tables from SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL
CREATE OR REPLACE VIEW CUSTOMER AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER;

CREATE OR REPLACE VIEW CUSTOMER_DEMOGRAPHICS AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER_DEMOGRAPHICS;

CREATE OR REPLACE VIEW DATE_DIM AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.DATE_DIM;

CREATE OR REPLACE VIEW ITEM AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.ITEM;

CREATE OR REPLACE VIEW STORE AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE;

CREATE OR REPLACE VIEW STORE_SALES AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE_SALES;

-- Query the Data
SHOW VIEWS;
SELECT * FROM CUSTOMER LIMIT 20;

### Part 2: Define the Semantic Model and Create a Semantic View
In this section, you will review a YAML-based semantic model and use it to create a Semantic View. You will also learn how to validate the YAML in the UI and via SQL.

### Review the YAML file
The YAML file has already been downloaded from the GitHub repository in `setup.sql`. In this step, you will verify that the file exists on the stage and review its contents.

In [ ]:
-- List files in the YAML stage
ls @YAML;

In [ ]:
import os
import yaml
import streamlit as st
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session

def read_yaml_from_stage(session: Session,
                         stage_path: str,
                         local_dir: str = "/tmp") -> dict:
    """
    Retrieve the YAML file from the stage to local, load it, and return its contents as a dict.

    :param session: Snowpark session
    :param stage_path: File path on the stage (e.g. "@MY_STAGE/configs/my_config.yaml")
    :param local_dir: Local directory to save the file (a writable area in the workspace)
    :return: YAML contents parsed into a dict
    """
    # → Fetch locally
    get_results = session.file.get(stage_path, local_dir)
    # get_results is a list containing info about the downloaded files
    if not get_results:
        raise FileNotFoundError(f"Failed to get file from stage: {stage_path}")
    # Normally get returns a list. Take the first result’s local file path
    local_path = get_results[0].file
    # Combine local_dir + the file name (if it’s not an absolute path)
    full_local_path = os.path.join(local_dir, local_path) if not os.path.isabs(local_path) else local_path

    # Load the YAML
    with open(full_local_path, 'r', encoding='utf-8') as f:
        yaml_data = yaml.safe_load(f)

    return yaml_data

session = get_active_session()
stage_yaml = "@TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW.yaml"
try:
    config = read_yaml_from_stage(session, stage_yaml, local_dir="/tmp")
    st.write(config)
except Exception as e:
    st.error("Error reading YAML:", e)

### (Optional) Create a Semantic View via SQL
You can create a Semantic View directly with SQL. This example mirrors the YAML-defined model so you can compare both approaches.

In [ ]:
USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;
CREATE OR REPLACE SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM
	tables (
		CUSTOMER primary key (C_CUSTOMER_SK),
		DATE as DATE_DIM primary key (D_DATE_SK),
		DEMO as CUSTOMER_DEMOGRAPHICS primary key (CD_DEMO_SK),
		ITEM primary key (I_ITEM_SK),
		STORE primary key (S_STORE_SK),
		STORESALES as STORE_SALES
        primary key (SS_SOLD_DATE_SK,SS_CDEMO_SK,SS_ITEM_SK,SS_STORE_SK,SS_CUSTOMER_SK)
	)
	relationships (
		SALESTOCUSTOMER as STORESALES(SS_CUSTOMER_SK) references CUSTOMER(C_CUSTOMER_SK),
		SALESTODATE as STORESALES(SS_SOLD_DATE_SK) references DATE(D_DATE_SK),
		SALESTODEMO as STORESALES(SS_CDEMO_SK) references DEMO(CD_DEMO_SK),
		SALESTOITEM as STORESALES(SS_ITEM_SK) references ITEM(I_ITEM_SK),
		SALETOSTORE as STORESALES(SS_STORE_SK) references STORE(S_STORE_SK)
	)
	facts (
		ITEM.COST as i_wholesale_cost,
		ITEM.PRICE as i_current_price,
		STORE.TAX_RATE as S_TAX_PRECENTAGE,
        STORESALES.SALES_QUANTITY as SS_QUANTITY
	)
	dimensions (
		CUSTOMER.BIRTHYEAR as C_BIRTH_YEAR,
		CUSTOMER.COUNTRY as C_BIRTH_COUNTRY,
		CUSTOMER.C_CUSTOMER_SK as c_customer_sk,
		DATE.DATE as D_DATE,
		DATE.D_DATE_SK as d_date_sk,
		DATE.MONTH as D_MOY,
		DATE.WEEK as D_WEEK_SEQ,
		DATE.YEAR as D_YEAR,
		DEMO.CD_DEMO_SK as cd_demo_sk,
		DEMO.CREDIT_RATING as CD_CREDIT_RATING,
		DEMO.MARITAL_STATUS as CD_MARITAL_STATUS,
		ITEM.BRAND as I_BRAND,
		ITEM.CATEGORY as I_CATEGORY,
		ITEM.CLASS as I_CLASS,
		ITEM.I_ITEM_SK as i_item_sk,
		STORE.MARKET as S_MARKET_ID,
		STORE.SQUAREFOOTAGE as S_FLOOR_SPACE,
		STORE.STATE as S_STATE,
		STORE.STORECOUNTRY as S_COUNTRY,
		STORE.S_STORE_SK as s_store_sk,
		STORESALES.SS_CDEMO_SK as ss_cdemo_sk,
		STORESALES.SS_CUSTOMER_SK as ss_customer_sk,
		STORESALES.SS_ITEM_SK as ss_item_sk,
		STORESALES.SS_SOLD_DATE_SK as ss_sold_date_sk,
		STORESALES.SS_STORE_SK as ss_store_sk
	)
	metrics (
		STORESALES.TOTALCOST as SUM(item.cost),
		STORESALES.TOTALSALESPRICE as SUM(SS_SALES_PRICE),
		STORESALES.TOTALSALESQUANTITY as SUM(SS_QUANTITY)
            WITH SYNONYMS = ( 'total sales quantity', 'total sales amount')
	)
;


In [ ]:
DESC SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM;

## Part 3: Convert Between YAML and Semantic Views
In this part, you will learn how to convert a Semantic View into its YAML representation and how to create a Semantic View from YAML. Snowflake supports both SQL functions and a Snowsight UI flow for these conversions.

### Method 1: Using Snowsight (UI)
Snowsight provides a no-code way to convert between a Semantic View and a YAML model.
1. Sign in to Snowsight.
2. Navigate to AI & ML → Cortex Analyst.
3. Select "Create new" → "Upload YAML file" to create from YAML, or open an existing Semantic View to export as YAML.
4. Follow the prompts to validate and save.


### Method 2: Using SQL Functions
The following examples demonstrate how to invoke the built-in functions to convert between YAML and Semantic Views directly from SQL.

### YAML to Semantic View

Reference: `system_create_semantic_view_from_yaml` (see Snowflake documentation).

**Note**: `SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML` cannot read a YAML file directly from a stage. In the next step, you will create a stored procedure that reads the YAML from the stage and then calls the function with the YAML text.

In [ ]:
CREATE OR REPLACE PROCEDURE CREATE_SEMANTIC_VIEW_FROM_STAGE_YAML(
    TARGET_SCHEMA STRING,          -- e.g. 'MY_DB.MY_SCHEMA' (must be fully-qualified)
    STAGE_FILE_URL STRING,         -- e.g. '@MY_DB.MY_SCHEMA.MY_STAGE/path/model.yaml'
    VERIFY_ONLY BOOLEAN DEFAULT FALSE
)
RETURNS STRING
LANGUAGE JAVASCRIPT
EXECUTE AS CALLER
AS
$$
function exec(sql, binds) {
  var s = snowflake.createStatement({ sqlText: sql, binds: binds || [] });
  return s.execute();
}

// Split '<DB>.<SCHEMA>'
function splitSchema(fqSchema) {
  var parts = (fqSchema || '').split('.');
  if (parts.length !== 2) {
    throw new Error("TARGET_SCHEMA must be fully qualified as 'DB.SCHEMA': " + fqSchema);
  }
  return { db: parts[0], schema: parts[1] };
}

// Extract the semantic view name from YAML (first top-level 'name:' line).
// Handles simple quoted or unquoted names. If not found, throws.
function extractViewNameFromYaml(yamlText) {
  if (!yamlText) throw new Error("YAML text is empty.");
  var re = /^[ \t]*name[ \t]*:[ \t]*(?:"([^"]+)"|'([^']+)'|([^\r\n#]+))/mi;
  var m = yamlText.match(re);
  if (!m) {
    throw new Error("Could not find a top-level 'name:' in the YAML specification.");
  }
  var raw = (m[1] || m[2] || m[3] || "").trim();
  // Remove inline trailing comments if present.
  raw = raw.replace(/[ \t]+#[^\r\n]*$/, "").trim();
  return raw;
}

var ctx = splitSchema(TARGET_SCHEMA);

// 1) Create a temporary file format to read YAML as a single column ($1).
var ff_name = 'FF_YAML_' + Date.now();
var ff_fqn  = ctx.db + '.' + ctx.schema + '.' + ff_name;

exec(
  'CREATE TEMPORARY FILE FORMAT ' + ff_fqn +
  ' TYPE = CSV FIELD_DELIMITER = NONE RECORD_DELIMITER = NONE'
);

// 2) Read YAML from stage using the named file format.
var yamlText;
try {
  var rs = exec(
    'SELECT $1 AS YAML_TEXT FROM ' + STAGE_FILE_URL + ' (FILE_FORMAT => ?) LIMIT 1',
    [ff_fqn]
  );
  if (!rs.next()) {
    throw new Error('Failed to read YAML from stage: ' + STAGE_FILE_URL);
  }
  yamlText = String(rs.getColumnValue(1));
} finally {
  // Best-effort cleanup (TEMP also drops at session end).
  try { exec('DROP FILE FORMAT IF EXISTS ' + ff_fqn); } catch (e) {}
}

// 3) Determine the target semantic view FQN from YAML.
var viewNameFromYaml = extractViewNameFromYaml(yamlText);
// Preserve quotes if the YAML used a quoted identifier with spaces.
var semanticViewFqn = ctx.db + '.' + ctx.schema + '.' + viewNameFromYaml;

// 4) Create or verify the semantic view from YAML.
//    Note: the system proc returns a status string; we return the FQN explicitly.
var callRs = exec(
  'CALL SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML(?, ?, ?)',
  [ (ctx.db + '.' + ctx.schema), yamlText, VERIFY_ONLY ]
);
callRs.next(); // status (not used further)

if (VERIFY_ONLY) {
  return 'VERIFIED: ' + semanticViewFqn;
} else {
  return 'CREATED: ' + semanticViewFqn;
}
$$;

In [ ]:
-- Create the Semantic View from the staged YAML
CALL CREATE_SEMANTIC_VIEW_FROM_STAGE_YAML(
  'TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL',
  '@TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW.yaml',
  FALSE  -- set to TRUE to validate only
);

In [ ]:
-- Inspect the created Semantic View
DESC SEMANTIC VIEW TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM

In [ ]:
select 
    get_ddl('SEMANTIC_VIEW',          'TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM', 
    true)

### Semantic View to YAML

Reference: `SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW` (see Snowflake documentation).

In [ ]:
-- Read the YAML representation of the Semantic View
SELECT SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW('TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM') AS yaml_format

In [ ]:
-- Save the YAML to an internal stage file
COPY INTO @TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW_2.yaml
FROM (
  SELECT SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW('TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM')
)
FILE_FORMAT = (
  TYPE = CSV
  FIELD_DELIMITER = NONE 
  RECORD_DELIMITER = NONE
  FIELD_OPTIONALLY_ENCLOSED_BY = NONE
  COMPRESSION = NONE
)
SINGLE = TRUE
OVERWRITE = TRUE;

In [ ]:
-- Verify the file was saved to the stage
ls @YAML

### Part 4: Querying a Semantic View
In this section, you will practice querying the Semantic View using dimensions, metrics, and filters expressed in business terms.

In [ ]:
-- Example query using the Semantic View syntax
SELECT * FROM SEMANTIC_VIEW
( 
 TPCDS_SEMANTIC_VIEW_SM
    DIMENSIONS 
            Item.Brand,
            Item.Category,            
            Date.Year,
            Date.Month,
            Store.State
    METRICS 
        StoreSales.TotalSalesQuantity
    WHERE
        Date.Year = '2002' AND Date.Month = '12' AND Store.State ='TX' AND Item.Category = 'Books'
) 
ORDER BY TotalSalesQuantity DESC LIMIT 10;

## Part 5: Integrate Semantic Views with Cortex Agents and Snowflake AI
In this part, you will learn how to connect your Semantic View to Cortex Agents and review tips to improve accuracy and performance when working with natural language queries.

First of all, you will create cortex agent object. There are two ways to generate it.

**Option1: SQL**

In [ ]:
CREATE OR REPLACE AGENT TKO25_SEMANTIC_VIEW_AGENT
WITH PROFILE='{ "display_name": "Snowflake TPCDS Agent" }'
    COMMENT=$$ This is an agent that can answer questions about TPCDS sample data. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "You are a data analyst who has access to Snowflake TPCDS.",
    "orchestration": "Use cortex analyst for detailed analysis.",
    "sample_questions": [
      {
        "question": "Show 10 sample records"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "cortex_analyst_snowflake_tpcds ",
        "description": "This is cortex analyst for revenue analysis."
      }
    }
  ],
  "tool_resources": {
    "cortex_analyst_snowflake_tpcds": {
      "semantic_view": "@TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM",
      "execution_environment": {
        "type": "warehouse",
        "warehouse": "TKP_APJ_SEMANTIC_VIEW_WH"
      }
    }
  }
}
$$

**Option2: Snowsight UI**
1. Sign in to Snowsight.
2. Navigate to AI & ML → Agents.
3. Click "Create Agent" and follow the guided setup.
4. Associate your Semantic View or upload a YAML model when prompted.

You will aceess cortex agents UI, and  Ask a question to Cortex Agents as below:

**Please show 10 sample records where the ITEM table has a BRAND equal to 'edu packunivamalg'.**

You may receive a response similar to the following from Cortex Agents:

---
The query executed successfully, but no records were found in the ITEM table with the exact brand name "edu packunivamalg". This typically means the requested value does not exist.

The query looked for exact matches in the BRAND column and found 0 rows. Consider the following possibilities:

- The brand name may not exist in this dataset.
- There may be spelling, capitalization, or formatting differences.
- The brand might appear as part of a longer string.

Next steps you can try:
1. Search for similar brand names (fuzzy match).
2. List a sample of distinct brand names from the ITEM table.
3. Search for brands that contain a partial match of this text.
---

In [ ]:
CREATE OR REPLACE TABLE ITEM_BRAND AS 
    SELECT DISTINCT I_BRAND FROM ITEM;

CREATE OR REPLACE CORTEX SEARCH SERVICE ITEM_BRAND_SEARCH_SERVICE
  ON I_BRAND
  WAREHOUSE = TKP_APJ_SEMANTIC_VIEW_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        I_BRAND,
    FROM ITEM_BRAND
);

In [ ]:
-- Add synonyms to improve natural language understanding in Cortex Agents

USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;

CREATE OR REPLACE SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM
	tables (
		CUSTOMER primary key (C_CUSTOMER_SK),
		DATE as DATE_DIM primary key (D_DATE_SK),
		DEMO as CUSTOMER_DEMOGRAPHICS primary key (CD_DEMO_SK),
		ITEM primary key (I_ITEM_SK),
		STORE primary key (S_STORE_SK),
		STORESALES as STORE_SALES
        primary key (SS_SOLD_DATE_SK,SS_CDEMO_SK,SS_ITEM_SK,SS_STORE_SK,SS_CUSTOMER_SK)
	)
    
	relationships (
		SALESTOCUSTOMER as STORESALES(SS_CUSTOMER_SK) references CUSTOMER(C_CUSTOMER_SK),
		SALESTODATE as STORESALES(SS_SOLD_DATE_SK) references DATE(D_DATE_SK),
		SALESTODEMO as STORESALES(SS_CDEMO_SK) references DEMO(CD_DEMO_SK),
		SALESTOITEM as STORESALES(SS_ITEM_SK) references ITEM(I_ITEM_SK),
		SALETOSTORE as STORESALES(SS_STORE_SK) references STORE(S_STORE_SK)
	)
    
	facts (
		ITEM.COST as i_wholesale_cost,
		ITEM.PRICE as i_current_price,
		STORE.TAX_RATE as S_TAX_PRECENTAGE,
        STORESALES.SALES_QUANTITY as SS_QUANTITY
	)
    
	dimensions (
		CUSTOMER.BIRTHYEAR as C_BIRTH_YEAR,
		CUSTOMER.COUNTRY as C_BIRTH_COUNTRY,
		CUSTOMER.C_CUSTOMER_SK as c_customer_sk,
		DATE.DATE as D_DATE,
		DATE.D_DATE_SK as d_date_sk,
		DATE.MONTH as D_MOY,
		DATE.WEEK as D_WEEK_SEQ,
		DATE.YEAR as D_YEAR,
		DEMO.CD_DEMO_SK as cd_demo_sk,
		DEMO.CREDIT_RATING as CD_CREDIT_RATING,
		DEMO.MARITAL_STATUS as CD_MARITAL_STATUS,

		-- Add semantic_search here to capture common user phrasing.
		ITEM.BRAND as I_BRAND
            with cortex search service ITEM_BRAND_SEARCH_SERVICE,

		ITEM.CATEGORY as I_CATEGORY,
		ITEM.CLASS as I_CLASS,
		ITEM.I_ITEM_SK as i_item_sk,
		STORE.MARKET as S_MARKET_ID,
		STORE.SQUAREFOOTAGE as S_FLOOR_SPACE,
		STORE.STATE as S_STATE,
		STORE.STORECOUNTRY as S_COUNTRY,
		STORE.S_STORE_SK as s_store_sk,
		STORESALES.SS_CDEMO_SK as ss_cdemo_sk,
		STORESALES.SS_CUSTOMER_SK as ss_customer_sk,
		STORESALES.SS_ITEM_SK as ss_item_sk,
		STORESALES.SS_SOLD_DATE_SK as ss_sold_date_sk,
		STORESALES.SS_STORE_SK as ss_store_sk
	)
    
	metrics (
		STORESALES.TOTALCOST as SUM(item.cost),
		STORESALES.TOTALSALESPRICE as SUM(SS_SALES_PRICE),
		STORESALES.TOTALSALESQUANTITY as SUM(SS_QUANTITY)
            WITH SYNONYMS = ('total sales quantity', 'total sales amount')
	)
;

[**Tips to improve performance for Semantic Model**]

Start with simple model tweaks (synonyms, samples) before adding Cortex Search or verified queries.

- ***Synonyms & descriptions***: Align column names and synonyms with user language; include key info like time zones.
- ***Multilingual synonyms***: Add synonyms in user languages (e.g., Japanese, Korean) for better query understanding.
- ***Sample values***: Provide examples to help the LLM link user terms (e.g., “Snowflake” vs “Snowflake Inc.”) to actual data.
- ***Cortex Search integration***: Use Search for columns with many unique values to handle typos and partial matches.
- ***Verified query repository***: Store trusted, complex SQL logic for reuse and onboarding suggestions.
- ***Custom instructions***: Use custom_instructions for clear behavioral rules; iterate gradually and test generated SQL.
- ***Semi-structured data***: Flatten when possible; otherwise, use fuzzy matching or VQR to extract nested elements.

## [Optional] Omni Free Trial
This optional section provides guidance on starting an Omni free trial so you can visualize and explore your data with your Semantic View.

To begin an Omni free trial:
1. Visit the Snowflake Marketplace in Snowsight.
2. Search for "Omni" and select the trial offering.
3. Follow the on-screen steps to provision and connect to your Snowflake account.
4. Point Omni to the schema containing your Semantic View to start exploring.

## [Optional] Share a Semantic View with Another Snowflake Account
This section outlines how to share your Semantic View securely with another Snowflake account.

Example steps:
1. Create or use a data share in your account.
2. Grant usage on the database and schema containing the Semantic View to the share.
3. Grant reference usage and necessary privileges on underlying objects.
4. In the consumer account, create a database from the provider's share and query the Semantic View.